# df_points with UNIX Timestamp #
## Creates df_points.csv with all points and idletimes #
## Checks the valid geo location
## deletes the invalid points: 


2019 - invalides: 52523 from 550865 that is 9.534640973741297 % - 04:38:48

2020 - invalides: 18082 from 662365 that is 2.729914775086244 %

2021 - ca.

### ca 1h for 1 year

In [370]:
def importdata(source):
    df = pd.read_csv(source)
    df = df[['bike_id', 'start_time', 'end_time', 'start_lat', 'start_lng', 'end_lat', 'end_lng']]
    return df

In [371]:
def dicOutOfDf(df01):
    import pandas as pd
    from tqdm import tqdm
    # Dict out of DataFrames of Trips per Bike

    # init DFs for every bike
    myDFs = {}

    for ind in tqdm(df01.index):
        if df01['bike_id'][ind] not in myDFs:
            myDFs[df01['bike_id'][ind]] = pd.DataFrame(
                columns=['bike_id', 'start_time', 'end_time', 'start_lat', 'start_lng', 'end_lat', 'end_lng'])
    return myDFs

In [372]:
def addRentalsToDf(df01,myDFs):
    from tqdm import tqdm
    # Add all rentals to their bike_id DF
    for ind in tqdm(df01.index):
        if df01['bike_id'][ind] in myDFs:
            myDFs[df01['bike_id'][ind]].loc[df01.index[ind]] = df01.iloc[ind]
    return myDFs

In [373]:
def sortDFs(myDFs):
    # sort every DF
    for df in myDFs.values():
        df[['bike_id', 'start_time','end_time']] = df[['bike_id', 'start_time','end_time']].astype(int)
        df[['start_lat','start_lng', 'end_lat', 'end_lng']] = df[['start_lat','start_lng', 'end_lat', 'end_lng']].astype(float)
        df.sort_values(by=['start_time'], inplace=True)
        df.reset_index(drop=True, inplace=True)
    return myDFs

In [374]:
def dfPoints(myDFs,radius):
    # DF of all points with stoodtime
    inval = 0
    val = 0 
    
    df_points = pd.DataFrame(columns=['bike_id', 'lat', 'lng', 'idle_time', 'time_start', 'time_end'])
    df_points[['bike_id', 'idle_time', 'time_start', 'time_end']] = df_points[['bike_id', 'idle_time', 'time_start', 'time_end']].astype(int)
    df_points[['lng', 'lat']] = df_points[['lng', 'lat']].astype(float)
    
    for df in tqdm(myDFs.values()):
        for ind in df.index[1:]:
            #in SECOUNDS
            _idle_time = (df['start_time'][ind] - df['end_time'][ind - 1]).astype(int)
            distance = qick_distance(df['end_lat'][ind-1], df['end_lng'][ind-1], df['start_lat'][ind], df['start_lng'][ind])
            if distance > radius:
                inval +=1
                continue
            val +=1
            dict = {'bike_id': df['bike_id'][ind], 'lng': df['start_lng'][ind], 'lat': df['start_lat'][ind],
                    'idle_time': _idle_time, 'time_start': df['end_time'][ind - 1], 'time_end': df['start_time'][ind]}
            df_points = df_points.append(dict, ignore_index=True)
    print(f"invalides: {inval} from {val} that is {inval/val * 100} %")
    return df_points

In [375]:
# returns distance in meter
# source: https://www.it-swarm.com.de/de/python/wie-kann-ich-die-entfernung-zwischen-zwei-punkten-breitengrad-laengengrad-schnell-schaetzen/1072488907/
def qick_distance(Lat1, Long1, Lat2, Long2):
    x = Lat2 - Lat1
    y = (Long2 - Long1)*cos((Lat2 + Lat1)*0.00872664626)  
    return 111.138*sqrt(x*x+y*y)*1000

In [376]:
def keplerMap(df_points):
    heatmap = KeplerGl()
    heatmap.add_data(data=df_points, name='points')
    heatmap.save_to_html(file_name='heatmap_test.html')

# Main 
## Create map

In [377]:
from keplergl import KeplerGl
import pandas as pd
pd.options.mode.chained_assignment = None
import time

ts1 = time.time()
source = '../data/raw_lendings/lendings_2018.csv'

df = importdata(source)
myDfs = dicOutOfDf(df)
myDfs = addRentalsToDf(df, myDfs)
myDfs = sortDFs(myDfs)


100%|██████████| 182199/182199 [04:50<00:00, 627.04it/s]


In [378]:
df.tail()

,bike_id,start_time,end_time,start_lat,start_lng,end_lat,end_lng
182194,71992,1546208583,1546209004,51.326871,12.374732,51.310257,12.373241
182195,71959,1546208955,1546209970,51.318458,12.377889,51.324615,12.336922
182196,41664,1546209503,1546210152,51.339337,12.379609,51.323619,12.335887
182197,72307,1546210213,1546210548,51.332060,12.371469,51.331342,12.358439
182198,72271,1546210428,1546210616,51.325764,12.344957,51.319218,12.344979


# Create DF_points
## without invalid points

In [379]:
from math import cos, sqrt
import pandas as pd
pd.options.mode.chained_assignment = None

radius = 20
df_points = dfPoints(myDfs,radius)

100%|██████████| 1300/1300 [05:50<00:00,  3.70it/s]

invalides: 41733 from 139166 that is 29.98792808588305 %


In [380]:
df_points.tail()

,bike_id,lat,lng,idle_time,time_start,time_end
139161,75861.0,51.338658,12.403918,11748.0,1.545331e+09,1.545343e+09
139162,75861.0,51.346151,12.396602,31244.0,1.545343e+09,1.545374e+09
139163,75861.0,51.340920,12.371810,138107.0,1.545375e+09,1.545513e+09
139164,75861.0,51.321733,12.342554,418348.0,1.545514e+09,1.545932e+09
139165,75861.0,0.000000,0.000000,87510.0,1.545934e+09,1.546021e+09


# Split Points at midnight

## save feature idle_time_next_day

In [381]:
df_points['idle_time'] = pd.to_timedelta(df_points['idle_time'],unit='s')
df_points['time_start'] = pd.to_datetime(df_points['time_start'],unit='s')
df_points['time_end'] = pd.to_datetime(df_points['time_end'],unit='s')

In [382]:
from tqdm import tqdm
from datetime import timedelta
import numpy as np

changed = 1
iterations = 0

df_points['idle_time_next_day'] = np.nan
df_points['over_night'] = 0

while changed:
    iterations += 1
    changed = 0
    for ind in tqdm(df_points.index):
        ts = df_points['time_start'][ind]
        te = df_points['time_end'][ind]
        t0 = ts.replace(hour=23, minute=59, second=59)
        
        # is timestood greater than same date 23:59:59
        #   -> over midnight
        if ts + df_points['idle_time'][ind] > t0:
            changed = 1
            
            underhang = t0 - ts
            overhang = te - t0
            
            # fix current day
            df_points['time_end'][ind] = ts.replace(hour=23, minute=59, second=59)
            df_points['idle_time'][ind] = underhang
            df_points['over_night'][ind] = 1
            df_points['idle_time_next_day'][ind] = overhang
            
            # add new row for the next day
            time_start = ts.replace(hour=0, minute=0, second=0)
            time_start += timedelta(days=1)
            flag = 0
            #if time_start + overhang > time_start.replace(hour=23, minute=59, second=59):
            #    flag = 1
            
            dict = {'bike_id': df_points['bike_id'][ind], 'lng': df_points['lng'][ind], 'lat': df_points['lat'][ind],
                    'idle_time': overhang, 'time_start': time_start, 'time_end': te,'over_night': flag, 'idle_time_next_day': np.nan}
            df_points = df_points.append(dict, ignore_index=True)

print(iterations)

100%|██████████| 215493/215493 [00:09<00:00, 22077.42it/s]

118


In [383]:
df_points.dtypes

bike_id                       float64
lat                           float64
lng                           float64
idle_time             timedelta64[ns]
time_start             datetime64[ns]
time_end               datetime64[ns]
idle_time_next_day             object
over_night                      int64
dtype: object

idle time in min

In [384]:
def idle_time_to_min(row):
    return row['idle_time'].seconds / 60

df_points['idle_time'] = df_points.apply(idle_time_to_min, axis=1)

add idle_time_next_day feature

In [385]:
df_points['idle_time_next_day'] = pd.to_timedelta(df_points['idle_time_next_day'])

def idle_time_next_day_to_min(row):
    if row['over_night'] == 1:
        t = row['idle_time_next_day'].total_seconds() / 60
        if t < 1439:
            return t
        else: return 1439
    else:
        return 0

df_points['idle_time_next_day'] = df_points.apply(idle_time_next_day_to_min, axis=1)


In [386]:
df_points

,bike_id,lat,lng,idle_time,time_start,time_end,idle_time_next_day,over_night
0,3694.0,51.323619,12.335887,12.833333,2017-12-31 23:47:09,2017-12-31 23:59:59,821.733333,1
1,3694.0,51.343813,12.381495,608.883333,2018-01-01 13:51:06,2018-01-01 23:59:59,877.033333,1
2,3694.0,51.343076,12.365177,35.850000,2018-01-02 14:55:53,2018-01-02 15:31:44,0.000000,0
3,3694.0,51.332874,12.373261,66.233333,2018-01-02 22:53:45,2018-01-02 23:59:59,999.333333,1
4,3694.0,51.297564,12.293015,424.933333,2018-01-03 16:55:03,2018-01-03 23:59:59,1439.000000,1
...,...,...,...,...,...,...,...,...
215488,64012.0,51.324559,12.338043,1439.983333,2018-08-30 00:00:00,2018-08-30 23:59:59,1439.000000,1
215489,64012.0,51.324559,12.338043,1439.983333,2018-08-31 00:00:00,2018-08-31 23:59:59,1439.000000,1
215490,64012.0,51.324559,12.338043,1439.983333,2018-09-01 00:00:00,2018-09-01 23:59:59,1439.000000,1
215491,64012.0,51.324559,12.338043,1439.983333,2018-09-02 00:00:00,2018-09-02 23:59:59,493.750000,1


change type to int

In [387]:
df_points['idle_time'] = df_points['idle_time'].astype(int)
df_points['idle_time_next_day'] = df_points['idle_time_next_day'].astype(int)
df_points

,bike_id,lat,lng,idle_time,time_start,time_end,idle_time_next_day,over_night
0,3694.0,51.323619,12.335887,12,2017-12-31 23:47:09,2017-12-31 23:59:59,821,1
1,3694.0,51.343813,12.381495,608,2018-01-01 13:51:06,2018-01-01 23:59:59,877,1
2,3694.0,51.343076,12.365177,35,2018-01-02 14:55:53,2018-01-02 15:31:44,0,0
3,3694.0,51.332874,12.373261,66,2018-01-02 22:53:45,2018-01-02 23:59:59,999,1
4,3694.0,51.297564,12.293015,424,2018-01-03 16:55:03,2018-01-03 23:59:59,1439,1
...,...,...,...,...,...,...,...,...
215488,64012.0,51.324559,12.338043,1439,2018-08-30 00:00:00,2018-08-30 23:59:59,1439,1
215489,64012.0,51.324559,12.338043,1439,2018-08-31 00:00:00,2018-08-31 23:59:59,1439,1
215490,64012.0,51.324559,12.338043,1439,2018-09-01 00:00:00,2018-09-01 23:59:59,1439,1
215491,64012.0,51.324559,12.338043,1439,2018-09-02 00:00:00,2018-09-02 23:59:59,493,1


# Save V2 df_points split

In [388]:
df_points['idle_time'].max()

1439

In [389]:
df_points.head()

,bike_id,lat,lng,idle_time,time_start,time_end,idle_time_next_day,over_night
0,3694.0,51.323619,12.335887,12,2017-12-31 23:47:09,2017-12-31 23:59:59,821,1
1,3694.0,51.343813,12.381495,608,2018-01-01 13:51:06,2018-01-01 23:59:59,877,1
2,3694.0,51.343076,12.365177,35,2018-01-02 14:55:53,2018-01-02 15:31:44,0,0
3,3694.0,51.332874,12.373261,66,2018-01-02 22:53:45,2018-01-02 23:59:59,999,1
4,3694.0,51.297564,12.293015,424,2018-01-03 16:55:03,2018-01-03 23:59:59,1439,1


## Add H3 Index

In [390]:
import h3

df_points['hex_id'] = 0

for ind in tqdm(df_points.index):
    df_points['hex_id'][ind] = h3.geo_to_h3(df_points['lat'][ind], df_points['lng'][ind], 8)
df_points.head()

100%|██████████| 215493/215493 [00:05<00:00, 42968.98it/s]


,bike_id,lat,lng,idle_time,time_start,time_end,idle_time_next_day,over_night,hex_id
0,3694.0,51.323619,12.335887,12,2017-12-31 23:47:09,2017-12-31 23:59:59,821,1,881f1a8d91fffff
1,3694.0,51.343813,12.381495,608,2018-01-01 13:51:06,2018-01-01 23:59:59,877,1,881f1a8cb1fffff
2,3694.0,51.343076,12.365177,35,2018-01-02 14:55:53,2018-01-02 15:31:44,0,0,881f1a8cbdfffff
3,3694.0,51.332874,12.373261,66,2018-01-02 22:53:45,2018-01-02 23:59:59,999,1,881f1a164bfffff
4,3694.0,51.297564,12.293015,424,2018-01-03 16:55:03,2018-01-03 23:59:59,1439,1,881f1a8da3fffff


# Add weather Data

In [391]:
dfw = pd.read_csv('../data/weather_data.csv')
dfw['datetime'] = pd.to_datetime(dfw['dt'], unit='s')
dfw = dfw.set_index(['datetime'])
dfw.head()

df_points['temp'] = np.nan
df_points['rain'] = np.nan
df_points['snow'] = np.nan

from tqdm import tqdm
pd.options.mode.chained_assignment = None

for ind in tqdm(df_points.index):
    wint = df_points['time_start'][ind].replace(minute=0,second=0)
    df_points['temp'][ind] = dfw['temp'][wint].copy()
    df_points['rain'][ind] = dfw['rain_1h'][wint].copy()
    df_points['snow'][ind] = dfw['snow_1h'][wint].copy()

df_points

100%|██████████| 215493/215493 [03:33<00:00, 1009.96it/s]


,bike_id,lat,lng,idle_time,time_start,time_end,idle_time_next_day,over_night,hex_id,temp,rain,snow
0,3694.0,51.323619,12.335887,12,2017-12-31 23:47:09,2017-12-31 23:59:59,821,1,881f1a8d91fffff,10.80,NaN,NaN
1,3694.0,51.343813,12.381495,608,2018-01-01 13:51:06,2018-01-01 23:59:59,877,1,881f1a8cb1fffff,8.52,NaN,NaN
2,3694.0,51.343076,12.365177,35,2018-01-02 14:55:53,2018-01-02 15:31:44,0,0,881f1a8cbdfffff,6.11,NaN,NaN
3,3694.0,51.332874,12.373261,66,2018-01-02 22:53:45,2018-01-02 23:59:59,999,1,881f1a164bfffff,2.96,NaN,NaN
4,3694.0,51.297564,12.293015,424,2018-01-03 16:55:03,2018-01-03 23:59:59,1439,1,881f1a8da3fffff,7.11,0.14,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...
215488,64012.0,51.324559,12.338043,1439,2018-08-30 00:00:00,2018-08-30 23:59:59,1439,1,881f1a8d93fffff,21.11,NaN,NaN
215489,64012.0,51.324559,12.338043,1439,2018-08-31 00:00:00,2018-08-31 23:59:59,1439,1,881f1a8d93fffff,14.11,NaN,NaN
215490,64012.0,51.324559,12.338043,1439,2018-09-01 00:00:00,2018-09-01 23:59:59,1439,1,881f1a8d93fffff,12.11,NaN,NaN
215491,64012.0,51.324559,12.338043,1439,2018-09-02 00:00:00,2018-09-02 23:59:59,493,1,881f1a8d93fffff,16.11,NaN,NaN


add unix timestamps

In [392]:
df_points['dt_start'] = pd.to_datetime(df_points['time_start']).map(pd.Timestamp.timestamp).astype(int)
df_points['dt_end'] = pd.to_datetime(df_points['time_end']).map(pd.Timestamp.timestamp).astype(int)
df_points['bike_id'] = df_points['bike_id'].astype(int)

# Add start time Feature

In [393]:
def add_start_min_feature(row):
    return ((row['dt_start'] % 86400) / 60).__round__(0)

df_points['start_min'] = df_points.apply(add_start_min_feature,axis=1)
df_points['start_min'] = df_points['start_min'].astype(int)

In [394]:
df_points

,bike_id,lat,lng,idle_time,time_start,time_end,idle_time_next_day,over_night,hex_id,temp,rain,snow,dt_start,dt_end,start_min
0,3694,51.323619,12.335887,12,2017-12-31 23:47:09,2017-12-31 23:59:59,821,1,881f1a8d91fffff,10.80,NaN,NaN,1514764029,1514764799,1427
1,3694,51.343813,12.381495,608,2018-01-01 13:51:06,2018-01-01 23:59:59,877,1,881f1a8cb1fffff,8.52,NaN,NaN,1514814666,1514851199,831
2,3694,51.343076,12.365177,35,2018-01-02 14:55:53,2018-01-02 15:31:44,0,0,881f1a8cbdfffff,6.11,NaN,NaN,1514904953,1514907104,896
3,3694,51.332874,12.373261,66,2018-01-02 22:53:45,2018-01-02 23:59:59,999,1,881f1a164bfffff,2.96,NaN,NaN,1514933625,1514937599,1374
4,3694,51.297564,12.293015,424,2018-01-03 16:55:03,2018-01-03 23:59:59,1439,1,881f1a8da3fffff,7.11,0.14,NaN,1514998503,1515023999,1015
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
215488,64012,51.324559,12.338043,1439,2018-08-30 00:00:00,2018-08-30 23:59:59,1439,1,881f1a8d93fffff,21.11,NaN,NaN,1535587200,1535673599,0
215489,64012,51.324559,12.338043,1439,2018-08-31 00:00:00,2018-08-31 23:59:59,1439,1,881f1a8d93fffff,14.11,NaN,NaN,1535673600,1535759999,0
215490,64012,51.324559,12.338043,1439,2018-09-01 00:00:00,2018-09-01 23:59:59,1439,1,881f1a8d93fffff,12.11,NaN,NaN,1535760000,1535846399,0
215491,64012,51.324559,12.338043,1439,2018-09-02 00:00:00,2018-09-02 23:59:59,493,1,881f1a8d93fffff,16.11,NaN,NaN,1535846400,1535932799,0


# Add day Feature

In [395]:
df_points.dtypes

bike_id                        int64
lat                          float64
lng                          float64
idle_time                      int64
time_start            datetime64[ns]
time_end              datetime64[ns]
idle_time_next_day             int64
over_night                     int64
hex_id                        object
temp                         float64
rain                         float64
snow                         float64
dt_start                       int64
dt_end                         int64
start_min                      int64
dtype: object

In [396]:
def add_day_feature(row):
    return row['time_start'].dayofweek

df_points['day'] = df_points.apply(add_day_feature,axis=1)

In [397]:
df_points

,bike_id,lat,lng,idle_time,time_start,time_end,idle_time_next_day,over_night,hex_id,temp,rain,snow,dt_start,dt_end,start_min,day
0,3694,51.323619,12.335887,12,2017-12-31 23:47:09,2017-12-31 23:59:59,821,1,881f1a8d91fffff,10.80,NaN,NaN,1514764029,1514764799,1427,6
1,3694,51.343813,12.381495,608,2018-01-01 13:51:06,2018-01-01 23:59:59,877,1,881f1a8cb1fffff,8.52,NaN,NaN,1514814666,1514851199,831,0
2,3694,51.343076,12.365177,35,2018-01-02 14:55:53,2018-01-02 15:31:44,0,0,881f1a8cbdfffff,6.11,NaN,NaN,1514904953,1514907104,896,1
3,3694,51.332874,12.373261,66,2018-01-02 22:53:45,2018-01-02 23:59:59,999,1,881f1a164bfffff,2.96,NaN,NaN,1514933625,1514937599,1374,1
4,3694,51.297564,12.293015,424,2018-01-03 16:55:03,2018-01-03 23:59:59,1439,1,881f1a8da3fffff,7.11,0.14,NaN,1514998503,1515023999,1015,2
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
215488,64012,51.324559,12.338043,1439,2018-08-30 00:00:00,2018-08-30 23:59:59,1439,1,881f1a8d93fffff,21.11,NaN,NaN,1535587200,1535673599,0,3
215489,64012,51.324559,12.338043,1439,2018-08-31 00:00:00,2018-08-31 23:59:59,1439,1,881f1a8d93fffff,14.11,NaN,NaN,1535673600,1535759999,0,4
215490,64012,51.324559,12.338043,1439,2018-09-01 00:00:00,2018-09-01 23:59:59,1439,1,881f1a8d93fffff,12.11,NaN,NaN,1535760000,1535846399,0,5
215491,64012,51.324559,12.338043,1439,2018-09-02 00:00:00,2018-09-02 23:59:59,493,1,881f1a8d93fffff,16.11,NaN,NaN,1535846400,1535932799,0,6


# Add month feature

In [398]:
def add_month_feature(row):
    return row['time_start'].strftime("%m")

df_points['month'] = df_points.apply(add_month_feature,axis=1)

In [399]:
#del df_points['time_start']
#del df_points['time_end']

Encode Hex_id

In [400]:
from sklearn import preprocessing

label_encoder = preprocessing.LabelEncoder()
df_points['hex_enc']= label_encoder.fit_transform(df_points['hex_id'])

Sort DF

In [401]:
df_points = df_points.reindex(columns=['bike_id', 'lat', 'lng', 'dt_start', 'dt_end', 'hex_id', 'hex_enc', 'temp', 'rain', 'snow', 'month', 'day', 'start_min', 'over_night', 'idle_time_next_day', 'idle_time'])
df_points['rain'] = df_points['rain'].fillna(0)
df_points['snow'] = df_points['snow'].fillna(0)

df_points.sort_values(by=['dt_start'])

,bike_id,lat,lng,dt_start,dt_end,hex_id,hex_enc,temp,rain,snow,month,day,start_min,over_night,idle_time_next_day,idle_time
0,3694,51.323619,12.335887,1514764029,1514764799,881f1a8d91fffff,195,10.80,0.0,0.0,12,6,1427,1,821,12
71,2508,51.323619,12.335887,1514764035,1514764799,881f1a8d91fffff,195,10.80,0.0,0.0,12,6,1427,1,1439,12
163,41851,51.332415,12.395158,1514764638,1514764799,881f1a1659fffff,81,10.80,0.0,0.0,12,6,1437,1,1439,2
139208,2508,51.323619,12.335887,1514764800,1514851199,881f1a8d91fffff,195,10.86,0.0,0.0,01,0,0,1,878,1439
139166,3694,51.323619,12.335887,1514764800,1514814103,881f1a8d91fffff,195,10.86,0.0,0.0,01,0,0,0,0,821
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
68370,41663,51.324645,12.311425,1546201232,1546204706,881f1a8d9dfffff,201,6.93,0.0,0.0,12,6,1221,0,0,57
133049,73851,51.343613,12.365640,1546202537,1546208319,881f1a8cbdfffff,170,6.93,0.0,0.0,12,6,1242,0,0,96
95332,72152,51.351505,12.407074,1546204570,1546205630,881f1ab96dfffff,268,6.36,0.0,0.0,12,6,1276,0,0,17
120442,72048,51.322000,12.373316,1546207062,1546207748,881f1a1643fffff,71,6.36,0.0,0.0,12,6,1318,0,0,11


In [402]:
df_points

,bike_id,lat,lng,dt_start,dt_end,hex_id,hex_enc,temp,rain,snow,month,day,start_min,over_night,idle_time_next_day,idle_time
0,3694,51.323619,12.335887,1514764029,1514764799,881f1a8d91fffff,195,10.80,0.00,0.0,12,6,1427,1,821,12
1,3694,51.343813,12.381495,1514814666,1514851199,881f1a8cb1fffff,164,8.52,0.00,0.0,01,0,831,1,877,608
2,3694,51.343076,12.365177,1514904953,1514907104,881f1a8cbdfffff,170,6.11,0.00,0.0,01,1,896,0,0,35
3,3694,51.332874,12.373261,1514933625,1514937599,881f1a164bfffff,75,2.96,0.00,0.0,01,1,1374,1,999,66
4,3694,51.297564,12.293015,1514998503,1515023999,881f1a8da3fffff,202,7.11,0.14,0.0,01,2,1015,1,1439,424
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
215488,64012,51.324559,12.338043,1535587200,1535673599,881f1a8d93fffff,196,21.11,0.00,0.0,08,3,0,1,1439,1439
215489,64012,51.324559,12.338043,1535673600,1535759999,881f1a8d93fffff,196,14.11,0.00,0.0,08,4,0,1,1439,1439
215490,64012,51.324559,12.338043,1535760000,1535846399,881f1a8d93fffff,196,12.11,0.00,0.0,09,5,0,1,1439,1439
215491,64012,51.324559,12.338043,1535846400,1535932799,881f1a8d93fffff,196,16.11,0.00,0.0,09,6,0,1,493,1439


In [403]:
dur = time.time() - ts1
print(dur)

3527.729553937912


In [404]:
ty_res = time.gmtime(dur)
res = time.strftime("%H:%M:%S",ty_res)
print(res)

00:58:47


# Save

In [405]:
df_points.to_csv('../data/df_points/df_points_2018.csv', index=False)

In [320]:
df_points

,bike_id,lat,lng,dt_start,dt_end,hex_id,hex_enc,temp,rain,snow,month,day,start_min,over_night,idle_time_next_day,idle_time
0,93513,51.317649,12.373610,1577833774,1577836799,881f1a1647fffff,79,2.13,0.0,0.0,12,1,1390,1,1439,50
1,93513,51.338133,12.385462,1577924396,1577961083,881f1a8cb7fffff,197,-1.11,0.0,0.0,01,3,20,0,0,611
2,93513,51.344889,12.373907,1577961466,1577968319,881f1a8cb1fffff,194,2.87,0.0,0.0,01,3,638,0,0,114
3,93513,51.343658,12.378786,1577968433,1577978806,881f1a8cb1fffff,194,6.54,0.0,0.0,01,3,754,0,0,172
4,93513,51.323373,12.335402,1577979896,1577990703,881f1a8d91fffff,229,4.29,0.0,0.0,01,3,945,0,0,180
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
883287,93544,51.321160,12.269497,1595289600,1595375999,881f1a8debfffff,264,15.89,0.0,0.0,07,1,0,1,1439,1439
883288,93544,51.321160,12.269497,1595376000,1595462399,881f1a8debfffff,264,13.39,0.0,0.0,07,2,0,1,1439,1439
883289,93544,51.321160,12.269497,1595462400,1595548799,881f1a8debfffff,264,15.08,0.0,0.0,07,3,0,1,1439,1439
883290,93544,51.321160,12.269497,1595548800,1595635199,881f1a8debfffff,264,15.34,0.0,0.0,07,4,0,1,1361,1439


In [4]:
import pandas as pd
df = pd.read_csv('../data/df_points/final_df_points_18_21.csv')
df = df.sort_values(by=['dt_start'])